# A notebook for creating sc graph animations with glimpses

In [ ]:
import napari
from cellpose import models
from octopuslite import utils, tile
import numpy as np
import os

def view(img):
    return napari.Viewer().add_image(img)
from napari_animation import Animation
from tqdm.auto import tqdm

import btrack
import dask.array as da

from pathlib import Path
import cv2

from skimage.transform import rescale, resize, downscale_local_mean
from skimage.io import imsave, imread
from scipy.ndimage.filters import gaussian_filter1d

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")
sns.set_palette("rocket_r")

def msd_calc(x1, y1, x2, y2):
    """
    Displacement calculation for cell movement between frames
    """
    return np.sqrt((x1-x2)**2+(y1-y2)**2)

def update_slider(event):
    # only trigger if update comes from first axis (optional)
        #ind_lambda = viewer.dims.indices[0]
    time = viewer.dims.current_step[0]
    viewer.text_overlay.text = f"{time:1.1f} hours"
text_size = 24
napari_scale = [1.4949402023919043e-07, 1.4949402023919043e-07]

import glob
from natsort import natsorted

label_text_size = 15
### glimpse size
size = 500
### resized images are to this scale
scale = 6048/1200

### Load all metadata

Both the image metadata and the assay layout metadata

In [ ]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
image_metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(image_metadata_fn)
assay_layout_metadata = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
assay_layout = utils.read_harmony_metadata(assay_layout_metadata, assay_layout=True)
assay_layout

# Iteratively load all tracks

and append to a track_dict dictionary

In [ ]:
tracks_dict = dict()
segmentation_dict = dict()
### iterate over all experimental conditions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):
    ### load tracks
    with btrack.dataio.HDF5FileHandler(
            f'/mnt/DATA/macrohet/segmentation/full_localisation/({row},{column}).h5',#
            'r', 
            obj_type = 'obj_type_1', 
            ) as hdf: 
            tracks = hdf.tracks
            seg = hdf.segmentation
            
    tracks_dict[(row, column)] = tracks
    segmentation_dict[(row, column)] = seg


# Compile all full length tracks into dataframe

Add extra information such as the MSD of cells between frames

In [ ]:
### list of track info dfs
dfs = list()
### empty dictionary for filtered tracks
filtered_tracks = dict()
### iterate over all tracks
for key in tracks_dict.keys():
    ### extract tracks only with max length
    filtered_tracks[key] = [track for track in tracks_dict[key] if len(track) == 75]
    ### iterate over full length tracks
    for track in filtered_tracks[key]:
        ### get info for assay layout
        info = assay_layout.loc[key]
        ### compile single track dictionary of info
        d = {'Time (hours)':track['t'], 
             'x':track['x'],
             'y':track['y'],
             'Area':track['area'], 
             'Intracellular Mtb content':track['mean_intensity-1'],
             'Mean Mtb content':[np.nanmean(track['mean_intensity-1']) for i in range(len(track['t']))],
             'Macroph. GFP expression':track['mean_intensity-0'],
             'Eccentricity':np.sqrt(1-((track['minor_axis_length']**2)/(track['major_axis_length']**2))),
             'MSD': [msd_calc(track['x'][i-1], 
                              track['y'][i-1], 
                              track['x'][i], 
                              track['y'][i]) 
                              if i != 0 else 0
                              for i in range(0, len(track))],
             'Strain':[info['Strain'] for i in range(len(track['t']))], 
             'Compound':[info['Compound'] for i in range(len(track['t']))], 
             'Concentration':[info['ConcentrationEC'] for i in range(len(track['t']))], 
             'Cell ID':[track.ID for i in range(len(track['t']))],
             'Acquisition ID':[key for i in range(len(track['t']))]}
        ### append df to list of dfs
        dfs.append(pd.DataFrame(d))
### concat single track dfs into big df
df = pd.concat(dfs, ignore_index=True)
### interpolate missing values as sometimes segmentation drops result in NaN
df.interpolate(inplace=True)

In [ ]:
df

In [ ]:
df

### Add category to discern initial amount of Mtb growth

Categorical

In [ ]:
initial_mtb = df.loc[df['Time (hours)'] == 0, 'Intracellular Mtb content']
initial_mtb_quartiles = pd.cut(initial_mtb, bins = [initial_mtb.quantile(.0), 
                                                    initial_mtb.quantile(.25), 
                                                    initial_mtb.quantile(0.5), 
                                                    initial_mtb.quantile(.75),
                                                    initial_mtb.quantile(1)], 
                                                    labels = ['Lower', 'Lower-mid', 'Upper-mid', 'Upper'])
df['Initial Mtb load (quartile)'] = initial_mtb_quartiles
df.fillna(method='ffill', inplace=True)

In [ ]:
final_mtb = df.loc[df['Time (hours)'] == 74, 'Intracellular Mtb content']
final_mtb_quartiles = pd.cut(final_mtb, bins = [final_mtb.quantile(.0), 
                                                    final_mtb.quantile(.25), 
                                                    final_mtb.quantile(0.5), 
                                                    final_mtb.quantile(.75),
                                                    final_mtb.quantile(1)], 
                                                    labels = ['Lower', 'Lower-mid', 'Upper-mid', 'Upper'])
df['Final Mtb load (quartile)'] = final_mtb_quartiles
df.fillna(method='bfill', inplace=True)

Now continuous assessment: ie. the actual value of the Mtb content in the first frame

In [ ]:
initial_mtb = df.loc[df['Time (hours)'] == 0, 'Intracellular Mtb content']
df['Initial Mtb load'] = initial_mtb
df.fillna(method='ffill', inplace=True)

In [ ]:
final_mtb = df.loc[df['Time (hours)'] == 74, 'Intracellular Mtb content']
df['Final Mtb load'] = final_mtb
df.fillna(method='bfill', inplace=True)

## Pick a single experiment to focus on?

In [ ]:
### select from conditions
selected_expt_df = df[(df['Compound'] == 'PZA') & (df['Concentration'] == 'EC50')]
acq_ID = selected_expt_df['Acquisition ID'].iloc[0]

In [ ]:
### get extreme cells
extreme_cases = pd.concat([selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.max(selected_expt_df['Mean Mtb content']))],
                           selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.min(selected_expt_df['Mean Mtb content']))]])
extreme_cases

In [ ]:
### isolate a single cell ID from df of interesting cells
ID = 300
### isolate sc df
sc_df = extreme_cases[extreme_cases['Cell ID']==ID]

In [ ]:
### or manually select from acquisition ID and Cell ID 
acq_ID = (3, 8)
selected_expt_df = df[(df['Acquisition ID'] == acq_ID)]
### get cell ID
ID = 300
### isolate sc df
sc_df = extreme_cases[extreme_cases['Cell ID']==ID]

# First assess glimpse with scale bars etc

In [ ]:
### use acq ID to pre load correct images
row, column = acq_ID
images = tile.compile_mosaic(
                             image_dir, 
                             metadata, 
                             row, 
                             column, 
                             set_plane = 'sum_proj',
                             ).astype(np.uint16)
### create base dirname
basedir = f'/mnt/DATA/macrohet/results/glimpses/labelled/{acq_ID}/{ID}/'
Path(os.path.dirname(basedir)).mkdir(parents=True, exist_ok=True)
### make glimpse stack of images
print(f'Creating glimpse ID: {acq_ID, ID}')
### create empty list for stack of images
glimpse_stack = list()
### iterate over time points from single cell 
for row in tqdm(sc_df.iterrows(), total = len(sc_df), desc = f'Creating glimpse ID: {acq_ID, ID}'):
    ### get coords
    t, x, y = row[1]['Time (hours)'], row[1]['y'], row[1]['x']
    ### select proper frame
    frame = images[t,...]
    ### scale as tracking was done on rescaled images
    x1, y1 = x*scale, y*scale
    ### create window for glimpse
    x1, x2, y1, y2 = x1, x1+size, y1, y1+size
    ### add padding for boundary cases
    frame = da.pad(frame, [(0, 0), (size/2, size/2), (size/2, size/2)], 'constant', constant_values = 0) 
    ### create glimpse image by cropping original image
    glimpse = frame[..., int(x1): int(x2), int(y1): int(y2)]
    ### append to glimpse stack
    glimpse_stack.append(glimpse)
    
### stack glimpse together
glimpse_stack = np.stack(glimpse_stack, axis = 1)
### load glimpse into memory
print(f'Loading glimpse stack {acq_ID, ID} into memory (can take several minutes)')
# glimpse_stack = glimpse_stack.compute().compute()

### make glimpse stack of masks
print(f'Creating mask glimpse ID: {acq_ID, ID}')
### get mask images
masks = segmentation_dict[acq_ID]
### create empty list for stack of images
mask_glimpse_stack = list()
coords_stack = list()
### iterate over time points from single cell 
for row in tqdm(sc_df.iterrows(), total = len(sc_df), desc = f'Creating mask glimpse ID: {acq_ID, ID}'):
    ### get coords
    t, x, y = row[1]['Time (hours)'], row[1]['y'], row[1]['x']
    ### select proper frame
    frame = masks[t,...]
    ### scale as tracking was done on rescaled images
    x1, y1 = x, y
    ### create window for glimpse
    x1, x2, y1, y2 = x1, x1+np.ceil(size/scale), y1, y1+np.ceil(size/scale)
    ### add padding for boundary cases
    frame = np.pad(frame, [(int(np.ceil(size/scale)/2), int(np.ceil(size/scale)/2)), (int(np.ceil(size/scale)/2), int(np.ceil(size/scale)/2))], 'constant', constant_values = 0) 
    ### create glimpse image by cropping original image
    mask_glimpse = frame[int(x1): int(x2), int(y1): int(y2)]
    ### select cell of interest
    mask_glimpse = mask_glimpse == mask_glimpse[int(np.ceil(size/scale)/2), int(np.ceil(size/scale)/2)]
    ### resize to image size
    mask_glimpse = cv2.resize(mask_glimpse.astype(np.uint8), (size,size))
    ### get contour
    cnts = cv2.findContours(mask_glimpse, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    coords = np.asarray([[t, i[0][1], i[0][0]] for i in cnts[0]])
    for c in cnts:
        cv2.drawContours(mask_glimpse, [c], -1, (255, 255, 255), thickness=5)
    ### change dtype of mask
    mask_glimpse = (mask_glimpse == 255).astype(np.uint16)
    ### append to mask glimpse stack
    mask_glimpse_stack.append(mask_glimpse)
    coords_stack.append(coords)
### stack mask glimpse together
mask_glimpse_stack = np.stack(mask_glimpse_stack, axis = 0)
### build coords into shape for napari
mask_shapes = np.asarray(coords_stack, )#axis = 0)
### get expt info
expt_info = sc_df['Strain'].iloc[0],sc_df['Compound'].iloc[0],sc_df['Concentration'].iloc[0], sc_df['Cell ID'].iloc[0]
expt_info = ','.join(map(str, expt_info))
### create a set of points to anchor labels at
label_points = [[t, 0, 500] for t in sc_df['Time (hours)']]
### create labels
text_overlay = f'cell ID:{acq_ID[0], acq_ID[1], ID}'
fixed_labels = {'macrophage':['Macrophage:green' for i in range(len(mask_shapes))],
                'mtb':['Mtb:magenta' for i in range(len(mask_shapes))],
                'mask':[f'Mask:cyan' for i in range(len(mask_shapes))],
                'info':[f'ID:{expt_info}' for i in range(len(mask_shapes))]}
### create lists of labels for tidier iteration of adding labels 
labels_list = list([fixed_labels['macrophage'], fixed_labels['mtb'], fixed_labels['mask'], fixed_labels['info']])
offset_list = list([[20,-5], [40,-5], [60,-5], [80,-5]])
colour_list = list(['#39FF14', 'magenta', 'cyan', 'white'])

# ### Check glimpse with labels
# viewer = napari.Viewer()
# viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], scale = napari_scale)
# viewer.theme = 'light'
# viewer.scale_bar.visible = True
# viewer.scale_bar.unit = 'm'
# viewer.scale_bar.font_size = text_size
# viewer.text_overlay.visible = True
# viewer.text_overlay.color = 'black'
# viewer.text_overlay.position = 'bottom_left'
# viewer.text_overlay.font_size = text_size
# viewer.dims.events.current_step.connect(update_slider)

# viewer.text_overlay.visible = True
# viewer.text_overlay.color = 'black'
# viewer.text_overlay.position = 'top_left'
# viewer.text_overlay.font_size = text_size
# viewer.text_overlay.text = 'test'

### new way of checking with overlaid labels etc
viewer = napari.Viewer()
viewer.add_image(mask_glimpse_stack, scale = napari_scale, colormap='cyan')
# # add the image
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], blending = ['additive', 'additive'],
                 scale = napari_scale)
# viewer.add_image(mask_glimpse_stack,)# channel_axis = 0, colormap= ['green', 'magenta'], blending = ['translucent', 'additive'],)# scale = napari_scale)
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

### other way of adding shape contours as mask outline
# viewer.add_shapes(
#     mask_shapes,
#     properties = properties,
#     scale = [1, napari_scale[0], napari_scale[1]],
# #     features = features,
#     shape_type='polygon',
#     edge_width=5,
#     edge_color='cyan',
# #     edge_color_cycle = edge_color_cycle,
#     face_color='transparent', 
#     name = 'mask shapes')

for config in zip(labels_list, offset_list, colour_list):
    string, offset, colour = config
    viewer.add_points(
    label_points, 
    scale = [1, napari_scale[0], napari_scale[1]],
    face_color = 'transparent', 
    edge_color = 'transparent', 
    text = {'string':string,
            'anchor': 'upper_right',
            'translation': offset,
            'size': text_size,
            'color': colour}, )

# Check view 

In [ ]:
time_overlay_font_size = 26
scale_bar_font_size = 26
label_font_size = 30
text_colour = 'white'

In [ ]:
zoom = viewer.camera.zoom
cam_coords = viewer.camera.center
### use this to assess camera angles before proceeding

# Then create .mp4 of glimpse and animated graphs

In [ ]:
### create glimpse .mp4 filename
mp4_fn = os.path.join(basedir, f'glimpse_{ID}.mp4')
# Path(os.path.dirname(mp4_fn)).mkdir(parents=True, exist_ok=True)
### launch napari and animate images into mp4
viewer = napari.Viewer()
viewer.add_image(mask_glimpse_stack, scale = napari_scale, colormap='cyan')
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], blending = 'additive', scale = napari_scale)
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = scale_bar_font_size
viewer.scale_bar.colored = True
viewer.scale_bar.color = text_colour
viewer.scale_bar.ticks = False
viewer.text_overlay.visible = True
viewer.text_overlay.color = text_colour
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = time_overlay_font_size
viewer.dims.events.current_step.connect(update_slider)
for config in zip(labels_list, offset_list, colour_list):
    string, offset, colour = config
    viewer.add_points(
    label_points, 
    scale = [1, napari_scale[0], napari_scale[1]],
    face_color = 'transparent', 
    edge_color = 'transparent', 
    text = {'string':string,
            'anchor': 'upper_right',
            'translation': offset,
            'size': label_font_size,
            'color': colour}, )
### initiate animation viewer
animation = Animation(viewer)
viewer.update_console({'animation': animation})
# viewer.camera.center = (0, 0, 3024, 3024)
viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
viewer.camera.zoom = zoom#*0.85
animation.capture_keyframe(steps = 100)
viewer.dims.current_step = (74.0,  cam_coords[-2], cam_coords[-1])
animation.capture_keyframe(steps = 100)
### Save glimpse as MP4
print(f'Saving glimpse .mp4 ID: {acq_ID, ID}')
animation.animate(mp4_fn, 
                  canvas_only=True,
                  fps = 5,
                  quality = 9)
viewer.close()
### save glimpse as a series of images
print(f'Creating glimpse image sequence ID: {acq_ID, ID}')
### create output directory
glimpse_seq_basedir = os.path.join(basedir, f'{ID}_glimpse_seq')
Path(glimpse_seq_basedir).mkdir(parents=True, exist_ok=True)
### create napari instances and use to save glimpse frames with scale bar and time
viewer = napari.Viewer()
viewer.add_image(mask_glimpse_stack, scale = napari_scale, colormap='cyan')
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], blending = 'additive', scale = napari_scale)
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = scale_bar_font_size
viewer.scale_bar.colored = True
viewer.scale_bar.color = text_colour
viewer.scale_bar.ticks = False
viewer.text_overlay.visible = True
viewer.text_overlay.color = text_colour
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = time_overlay_font_size
viewer.dims.events.current_step.connect(update_slider)
for config in zip(labels_list, offset_list, colour_list):
    string, offset, colour = config
    viewer.add_points(
    label_points, 
    scale = [1, napari_scale[0], napari_scale[1]],
    face_color = 'transparent', 
    edge_color = 'transparent', 
    text = {'string':string,
            'anchor': 'upper_right',
            'translation': offset,
            'size': label_font_size,
            'color': colour}, )
### save image sequence
for t in tqdm(list(sc_df['Time (hours)']), desc = f'Saving glimpse image sequence ID: {acq_ID, ID}'):
    viewer.dims.current_step = (t, cam_coords[-2], cam_coords[-1])
    viewer.camera.zoom = zoom#*0.85
    glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f'glimpse_{ID}_t_{t}.tiff')
    imsave(glimpse_seq_fn, viewer.screenshot())
viewer.close()
### create sequence of graph images
print(f'Creating graph image sequence ID: {acq_ID, ID}')
# create graph output folder
glimpse_graph_seq_basedir = os.path.join(basedir, f'{ID}_glimpse_graph_seq')
Path(glimpse_graph_seq_basedir).mkdir(parents = True, exist_ok = True)
### create graph by interpolating missing y values
y = sc_df['Intracellular Mtb content'].interpolate().values
x = sc_df['Time (hours)'].values
### smooth graph
y = gaussian_filter1d(y, sigma=1.5)
### iterate over time points
for n, row in tqdm(enumerate(sc_df.iterrows()), total = len(sc_df), 
                   desc = f'Saving graph image sequence ID: {acq_ID, ID}'):
    ### select portion of data to plot up to current time point iteration
    plot_x = sc_df['Time (hours)'].iloc[0:n+1].values
    plot_y = y[0:n+1]
    ### get time ID for labelling purposes
    t = row[1]['Time (hours)']     
    ### initiate plot with two components
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5), gridspec_kw={'width_ratios': [1, 1]})
    ### label fn with time point
    glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f'glimpse_{ID}_t_{t}.tiff')
    ### load glimpse image from this timepoint
    glimpse_img = imread(glimpse_seq_fn)
    ### add glimpse frame to first part of subplot
    ax1.imshow(glimpse_img)
    ### turn axis off for glimpse
    ax1.axis('off')
    ### plot portion of graph 
    ax2.plot(plot_x, plot_y, c = 'magenta')
    ### set aspect
    # ax2.set_aspect(75/y.max())
    ### label graph properly 
    ax2.set(xlabel = 'Time (hours)', 
            ylabel = f'Mtb content of cell ID: {ID} \n (raw fluorescence intensity)', 
            ylim=(y.min()*0.85,y.max()*1.15), 
            xlim = (0,75))
    ### despine plot
    sns.despine()
    ### new fn for joint glimpse plot
    new_graph_fn = os.path.join(glimpse_graph_seq_basedir, f'glimpse_graph_{ID}_t_{t}.tiff')
    plt.savefig(new_graph_fn, bbox_inches = 'tight', dpi = 314)

## Compile single frame graphs into stack for animation
print(f'Creating graph and glimpse image sequence ID: {acq_ID, ID}')
plots = list()
for fn in natsorted(glob.glob(os.path.join(glimpse_graph_seq_basedir, '*.tiff'))):
    plot = imread(fn)
    plots.append(plot)
plots = np.stack(plots, axis = 0)
### create final fn
glimpse_graph_fn = os.path.join(basedir, f'glimpse_graph_{ID}.mp4')
### initiate napari session for animation
viewer = napari.Viewer()
viewer.add_image(plots)#, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
viewer.theme = 'light'
# viewer.scale_bar.visible = True
# viewer.scale_bar.unit = 'm'
# viewer.scale_bar.font_size = text_size
# viewer.text_overlay.visible = True
# viewer.text_overlay.color = 'black'
# viewer.text_overlay.position = 'bottom_left'
# viewer.text_overlay.font_size = text_size
# viewer.dims.events.current_step.connect(update_slider)
animation = Animation(viewer)
viewer.update_console({'animation': animation})
# # viewer.camera.center = (0, 0, 3024, 3024)
viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
# viewer.camera.zoom = 0.23
animation.capture_keyframe(steps = 100)
viewer.dims.current_step = (75.0,  cam_coords[-2], cam_coords[-1])
animation.capture_keyframe(steps = 100)
print(f'Saving graph and glimpse animation ID: {acq_ID, ID}')
animation.animate(glimpse_graph_fn, 
                  canvas_only=True,
                  fps = 5,
                  quality = 9)
viewer.close()
# plt.clf()

# Iteratively generate glimpses

In [ ]:
if acq_ID not in [(4, 9), (4, 6), (5, 9)]

In [ ]:
im = imread('/mnt/DATA/macrohet/results/glimpses/labelled/(4, 9)/264/264_glimpse_seq/glimpse_264_t_0.tiff')

In [ ]:
im.shape

In [ ]:
plt.imshow(im[...,0], cmap = 'Reds_r')

In [ ]:
plt.imshow(im[...,1], cmap = 'Greens_r')

In [ ]:
plt.imshow(im[...,2], cmap = 'Blues_r')

In [ ]:
plt.imshow(im[...,3])

In [ ]:
np.mean(im[...,3])

In [ ]:
np.mean(im2[...,3])

In [ ]:
plt.imshow(im2[...,3])

In [ ]:
im2.shape

In [ ]:
im2 = imread('/mnt/DATA/macrohet/results/glimpses/labelled/(4, 9)/17/17_glimpse_seq/glimpse_17_t_0.tiff')

In [ ]:
os.path.join(basedir, f'{ID}_glimpse_seq', f'glimpse_{ID}_t_{t}.tiff')

In [ ]:
for n, assay_info in tqdm(enumerate(assay_layout.iterrows()), 
                       total = len(assay_layout), 
                       desc = 'Iterating over different assays, generating two extreme glimpses per assay'):
    
                          
    ### pull acq ID from assay info
    acq_ID = assay_info[0]

    ### iterate over different experiments
    selected_expt_df = df[(df['Acquisition ID'] == acq_ID)]

    ### get extreme cells
    extreme_cases = pd.concat([selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.max(selected_expt_df['Mean Mtb content']))],
                               selected_expt_df[(selected_expt_df['Mean Mtb content'] == np.min(selected_expt_df['Mean Mtb content']))]])
    
    ### iterate over two extreme cases in either
    for ID in tqdm(extreme_cases['Cell ID'].unique(), 
                   desc = f'Iterating over two extreme cells in acquisition ID:{acq_ID}'):
        ### isolate a single cell ID from df of interesting cells
        sc_df = extreme_cases[extreme_cases['Cell ID']==ID]
        
        # ### get cell ID
        # ID = sc_df['Cell ID'].iloc[0]
        # ### get acquisition ID 
        # acq_ID = sc_df['Acquisition ID'].iloc[0]
        ### use acq ID to pre load correct images
        row, column = acq_ID
        images = tile.compile_mosaic(
                                     image_dir, 
                                     metadata, 
                                     row, 
                                     column, 
                                     set_plane = 'sum_proj',
                                     ).astype(np.uint16)
        ### create base dirname
        basedir = f'/mnt/DATA/macrohet/results/glimpses/labelled/{acq_ID}/{ID}/'
        Path(os.path.dirname(basedir)).mkdir(parents=True, exist_ok=True)
        
        ### skip certain pos
        ### find average value of 4th channel in first image
        ### test fn 
        test_fn = os.path.join(basedir, f'{ID}_glimpse_seq', f'glimpse_{ID}_t_{0}.tiff')
        im = imread(test_fn)
        mean = np.mean(im[...,3])
        if mean >= 255.0:
            print(f'Skipping {acq_ID, ID} as seems to already be properly rendered')
            continue
        else: 
            print(f'Continuing {acq_ID, ID} as seems to already be falsely rendered')
                    
        
        ### make glimpse stack of images
        print(f'Creating glimpse ID: {acq_ID, ID}')
        ### create empty list for stack of images
        glimpse_stack = list()
        ### iterate over time points from single cell 
        for row in tqdm(sc_df.iterrows(), total = len(sc_df), desc = f'Creating glimpse ID: {acq_ID, ID}'):
            ### get coords
            t, x, y = row[1]['Time (hours)'], row[1]['y'], row[1]['x']
            ### select proper frame
            frame = images[t,...]
            ### scale as tracking was done on rescaled images
            x1, y1 = x*scale, y*scale
            ### create window for glimpse
            x1, x2, y1, y2 = x1, x1+size, y1, y1+size
            ### add padding for boundary cases
            frame = da.pad(frame, [(0, 0), (size/2, size/2), (size/2, size/2)], 'constant', constant_values = 0) 
            ### create glimpse image by cropping original image
            glimpse = frame[..., int(x1): int(x2), int(y1): int(y2)]
            ### append to glimpse stack
            glimpse_stack.append(glimpse)

        ### stack glimpse together
        glimpse_stack = np.stack(glimpse_stack, axis = 1)
        ### load glimpse into memory
        print(f'Loading glimpse stack {acq_ID, ID} into memory (can take several minutes)')
        glimpse_stack = glimpse_stack.compute().compute()

        ### make glimpse stack of masks
        print(f'Creating mask glimpse ID: {acq_ID, ID}')
        ### get mask images
        masks = segmentation_dict[acq_ID]
        ### create empty list for stack of images
        mask_glimpse_stack = list()
        coords_stack = list()
        ### iterate over time points from single cell 
        for row in tqdm(sc_df.iterrows(), total = len(sc_df), desc = f'Creating mask glimpse ID: {acq_ID, ID}'):
            ### get coords
            t, x, y = row[1]['Time (hours)'], row[1]['y'], row[1]['x']
            ### select proper frame
            frame = masks[t,...]
            ### scale as tracking was done on rescaled images
            x1, y1 = x, y
            ### create window for glimpse
            x1, x2, y1, y2 = x1, x1+np.ceil(size/scale), y1, y1+np.ceil(size/scale)
            ### add padding for boundary cases
            frame = np.pad(frame, [(int(np.ceil(size/scale)/2), int(np.ceil(size/scale)/2)), (int(np.ceil(size/scale)/2), int(np.ceil(size/scale)/2))], 'constant', constant_values = 0) 
            ### create glimpse image by cropping original image
            mask_glimpse = frame[int(x1): int(x2), int(y1): int(y2)]
            ### select cell of interest
            mask_glimpse = mask_glimpse == mask_glimpse[int(np.ceil(size/scale)/2), int(np.ceil(size/scale)/2)]
            ### resize to image size
            mask_glimpse = cv2.resize(mask_glimpse.astype(np.uint8), (size,size))
            ### get contour
            cnts = cv2.findContours(mask_glimpse, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            coords = np.asarray([[t, i[0][1], i[0][0]] for i in cnts[0]])
            for c in cnts:
                cv2.drawContours(mask_glimpse, [c], -1, (255, 255, 255), thickness=5)
            ### change dtype of mask
            mask_glimpse = (mask_glimpse == 255).astype(np.uint16)
            ### append to mask glimpse stack
            mask_glimpse_stack.append(mask_glimpse)
            coords_stack.append(coords)
        ### stack mask glimpse together
        mask_glimpse_stack = np.stack(mask_glimpse_stack, axis = 0)
        ### build coords into shape for napari
        mask_shapes = np.asarray(coords_stack, )#axis = 0)
        ### get expt info
        expt_info = sc_df['Strain'].iloc[0],sc_df['Compound'].iloc[0],sc_df['Concentration'].iloc[0], sc_df['Cell ID'].iloc[0]
        expt_info = ','.join(map(str, expt_info))
        ### create a set of points to anchor labels at
        label_points = [[t, 0, 500] for t in sc_df['Time (hours)']]
        ### create labels
        text_overlay = f'cell ID:{acq_ID[0], acq_ID[1], ID}'
        fixed_labels = {'macrophage':['Macrophage:green' for i in range(len(mask_shapes))],
                        'mtb':['Mtb:magenta' for i in range(len(mask_shapes))],
                        'mask':[f'Mask:cyan' for i in range(len(mask_shapes))],
                        'info':[f'ID:{expt_info}' for i in range(len(mask_shapes))]}
        ### create lists of labels for tidier iteration of adding labels 
        labels_list = list([fixed_labels['macrophage'], fixed_labels['mtb'], fixed_labels['mask'], fixed_labels['info']])
        offset_list = list([[20,-5], [40,-5], [60,-5], [80,-5]])
        colour_list = list(['#39FF14', 'magenta', 'cyan', 'white'])

        ### create glimpse .mp4 filename
        mp4_fn = os.path.join(basedir, f'glimpse_{ID}.mp4')
        # Path(os.path.dirname(mp4_fn)).mkdir(parents=True, exist_ok=True)
        ### launch napari and animate images into mp4
        viewer = napari.Viewer()
        viewer.add_image(mask_glimpse_stack, scale = napari_scale, colormap='cyan')
        viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], blending = 'additive', scale = napari_scale)
        viewer.theme = 'light'
        viewer.scale_bar.visible = True
        viewer.scale_bar.unit = 'm'
        viewer.scale_bar.font_size = scale_bar_font_size
        viewer.scale_bar.colored = True
        viewer.scale_bar.color = text_colour
        viewer.scale_bar.ticks = False
        viewer.text_overlay.visible = True
        viewer.text_overlay.color = text_colour
        viewer.text_overlay.position = 'bottom_left'
        viewer.text_overlay.font_size = time_overlay_font_size
        viewer.dims.events.current_step.connect(update_slider)
        for config in zip(labels_list, offset_list, colour_list):
            string, offset, colour = config
            viewer.add_points(
            label_points, 
            scale = [1, napari_scale[0], napari_scale[1]],
            face_color = 'transparent', 
            edge_color = 'transparent', 
            text = {'string':string,
                    'anchor': 'upper_right',
                    'translation': offset,
                    'size': label_font_size,
                    'color': colour}, )
        ### initiate animation viewer
        animation = Animation(viewer)
        viewer.update_console({'animation': animation})
        # viewer.camera.center = (0, 0, 3024, 3024)
        viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
        viewer.camera.zoom = zoom#*0.85
        animation.capture_keyframe(steps = 100)
        viewer.dims.current_step = (74.0,  cam_coords[-2], cam_coords[-1])
        animation.capture_keyframe(steps = 100)
        ### Save glimpse as MP4
        print(f'Saving glimpse .mp4 ID: {acq_ID, ID}')
        animation.animate(mp4_fn, 
                          canvas_only=True,
                          fps = 5,
                          quality = 9)
        viewer.close()
        ### save glimpse as a series of images
        print(f'Creating glimpse image sequence ID: {acq_ID, ID}')
        ### create output directory
        glimpse_seq_basedir = os.path.join(basedir, f'{ID}_glimpse_seq')
        Path(glimpse_seq_basedir).mkdir(parents=True, exist_ok=True)
        ### create napari instances and use to save glimpse frames with scale bar and time
        viewer = napari.Viewer()
        viewer.add_image(mask_glimpse_stack, scale = napari_scale, colormap='cyan')
        viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], blending = 'additive', scale = napari_scale)
        viewer.theme = 'light'
        viewer.scale_bar.visible = True
        viewer.scale_bar.unit = 'm'
        viewer.scale_bar.font_size = scale_bar_font_size
        viewer.scale_bar.colored = True
        viewer.scale_bar.color = text_colour
        viewer.scale_bar.ticks = False
        viewer.text_overlay.visible = True
        viewer.text_overlay.color = text_colour
        viewer.text_overlay.position = 'bottom_left'
        viewer.text_overlay.font_size = time_overlay_font_size
        viewer.dims.events.current_step.connect(update_slider)
        for config in zip(labels_list, offset_list, colour_list):
            string, offset, colour = config
            viewer.add_points(
            label_points, 
            scale = [1, napari_scale[0], napari_scale[1]],
            face_color = 'transparent', 
            edge_color = 'transparent', 
            text = {'string':string,
                    'anchor': 'upper_right',
                    'translation': offset,
                    'size': label_font_size,
                    'color': colour}, )
        ### save image sequence
        for t in tqdm(list(sc_df['Time (hours)']), desc = f'Saving glimpse image sequence ID: {acq_ID, ID}'):
            viewer.dims.current_step = (t, cam_coords[-2], cam_coords[-1])
            viewer.camera.zoom = zoom#*0.85
            glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f'glimpse_{ID}_t_{t}.tiff')
            imsave(glimpse_seq_fn, viewer.screenshot())
        viewer.close()
        ### create sequence of graph images
        print(f'Creating graph image sequence ID: {acq_ID, ID}')
        # create graph output folder
        glimpse_graph_seq_basedir = os.path.join(basedir, f'{ID}_glimpse_graph_seq')
        Path(glimpse_graph_seq_basedir).mkdir(parents = True, exist_ok = True)
        ### create graph by interpolating missing y values
        y = sc_df['Intracellular Mtb content'].interpolate().values
        x = sc_df['Time (hours)'].values
        ### smooth graph
        y = gaussian_filter1d(y, sigma=1.5)
        ### iterate over time points
        for n, row in tqdm(enumerate(sc_df.iterrows()), total = len(sc_df), 
                           desc = f'Saving graph image sequence ID: {acq_ID, ID}'):
            ### select portion of data to plot up to current time point iteration
            plot_x = sc_df['Time (hours)'].iloc[0:n+1].values
            plot_y = y[0:n+1]
            ### get time ID for labelling purposes
            t = row[1]['Time (hours)']     
            ### initiate plot with two components
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5), gridspec_kw={'width_ratios': [1, 1]})
            ### label fn with time point
            glimpse_seq_fn = os.path.join(glimpse_seq_basedir, f'glimpse_{ID}_t_{t}.tiff')
            ### load glimpse image from this timepoint
            glimpse_img = imread(glimpse_seq_fn)
            ### add glimpse frame to first part of subplot
            ax1.imshow(glimpse_img)
            ### turn axis off for glimpse
            ax1.axis('off')
            ### plot portion of graph 
            ax2.plot(plot_x, plot_y, c = 'magenta')
            ### set aspect
            # ax2.set_aspect(75/y.max())
            ### label graph properly 
            ax2.set(xlabel = 'Time (hours)', 
                    ylabel = f'Mtb content of cell ID: {ID} \n (raw fluorescence intensity)', 
                    ylim=(y.min()*0.85,y.max()*1.15), 
                    xlim = (0,75))
            ### despine plot
            sns.despine()
            ### new fn for joint glimpse plot
            new_graph_fn = os.path.join(glimpse_graph_seq_basedir, f'glimpse_graph_{ID}_t_{t}.tiff')
            plt.savefig(new_graph_fn, bbox_inches = 'tight', dpi = 314)

        ## Compile single frame graphs into stack for animation
        print(f'Creating graph and glimpse image sequence ID: {acq_ID, ID}')
        plots = list()
        for fn in natsorted(glob.glob(os.path.join(glimpse_graph_seq_basedir, '*.tiff'))):
            plot = imread(fn)
            plots.append(plot)
        plots = np.stack(plots, axis = 0)
        ### create final fn
        glimpse_graph_fn = os.path.join(basedir, f'glimpse_graph_{ID}.mp4')
        ### initiate napari session for animation
        viewer = napari.Viewer()
        viewer.add_image(plots)#, channel_axis = 0, colormap= ['green', 'magenta'], scale = scale
        viewer.theme = 'light'
        # viewer.scale_bar.visible = True
        # viewer.scale_bar.unit = 'm'
        # viewer.scale_bar.font_size = text_size
        # viewer.text_overlay.visible = True
        # viewer.text_overlay.color = 'black'
        # viewer.text_overlay.position = 'bottom_left'
        # viewer.text_overlay.font_size = text_size
        # viewer.dims.events.current_step.connect(update_slider)
        animation = Animation(viewer)
        viewer.update_console({'animation': animation})
        # # viewer.camera.center = (0, 0, 3024, 3024)
        viewer.dims.current_step = (0, cam_coords[-2], cam_coords[-1])
        # viewer.camera.zoom = 0.23
        animation.capture_keyframe(steps = 100)
        viewer.dims.current_step = (75.0,  cam_coords[-2], cam_coords[-1])
        animation.capture_keyframe(steps = 100)
        print(f'Saving graph and glimpse animation ID: {acq_ID, ID}')
        animation.animate(glimpse_graph_fn, 
                          canvas_only=True,
                          fps = 5,
                          quality = 9)
        viewer.close()
        # plt.clf()

# Check view

In [ ]:
help(viewer.scale_bar)

In [ ]:
time_overlay_font_size = 26
scale_bar_font_size = 20
label_font_size = 30

In [ ]:
viewer.camera.zoom = zoom*0.85
viewer = napari.Viewer()
viewer.add_image(mask_glimpse_stack, scale = napari_scale, colormap='cyan')
viewer.add_image(glimpse_stack, channel_axis = 0, colormap= ['green', 'magenta'], blending = 'additive', scale = napari_scale)
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.ticks = False
viewer.scale_bar.font_size = scale_bar_font_size
viewer.scale_bar.position = 'bottom_right'

viewer.text_overlay.visible = True
viewer.text_overlay.color = 'black'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = time_overlay_font_size
viewer.dims.events.current_step.connect(update_slider)
for config in zip(labels_list, offset_list, colour_list):
    string, offset, colour = config
    viewer.add_points(
    label_points, 
    scale = [1, napari_scale[0], napari_scale[1]],
    face_color = 'transparent', 
    edge_color = 'transparent', 
    text = {'string':string,
            'anchor': 'upper_right',
            'translation': offset,
            'size': label_font_size,
            'color': colour}, )

# Alternate way of compiling mp4

In [ ]:
mp4_files = [os.path.join(root, f) for root, dirs, files in os.walk('/mnt/DATA/macrohet/results/glimpses/labelled/') for f in files if f.endswith('.mp4') and 'hq' not in f]


In [ ]:
mp4_files

In [ ]:
import cv2
import os


for file in tqdm(mp4_files):
    
    # Split the file path into directory, filename, and extension
    directory, filename_ext = os.path.split(file)
    filename, ext = os.path.splitext(filename_ext)

    # Extract the last numbers from the filename
    last_numbers = "".join(filter(str.isdigit, filename))

    # Build the new filename and file path
    new_filename = f"{last_numbers}_{filename.replace(last_numbers, '')}seq"
    new_path = os.path.join(directory, new_filename)   
    
    # Set the directory path containing the images
    image_folder = new_path

    # Set the output video file name
    video_name = os.path.join(directory, f'hq_{filename}.mp4')
    
    if os.path.exists(video_name):
        continue

    # Set the frame rate of the output video
    frame_rate = 3

    # Get the list of images in the directory
    images = [img for img in os.listdir(image_folder) if img.endswith('.tiff')]

    # Sort the images in alphabetical order
    images.sort()

    # Get the first image to get the size of the video
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, channels = frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(video_name, fourcc, frame_rate, (width, height))

    # Loop through the images and add them to the video
    for image in images:
        img_path = os.path.join(image_folder, image)
        frame = cv2.imread(img_path)

        # Write the frame to the video
        video_writer.write(frame)

    # Release the VideoWriter object
    video_writer.release()

In [ ]:
# Set the directory path containing the images
image_folder = '/mnt/DATA/macrohet/results/glimpses/labelled/multi_plot/(3, 8)/300/300_glimpse_seq'

# Set the output video file name
video_name = '/mnt/DATA/macrohet/results/glimpses/labelled/multi_plot/(3, 8)/300/hq_glimpse_300.mp4'

# Set the frame rate of the output video
frame_rate = 3

# Get the list of images in the directory
images = [img for img in os.listdir(image_folder) if img.endswith('.tiff')]

# Sort the images in alphabetical order
images.sort()

# Get the first image to get the size of the video
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, channels = frame.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_name, fourcc, frame_rate, (width, height))

# Loop through the images and add them to the video
for image in images:
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)

    # Write the frame to the video
    video_writer.write(frame)

# Release the VideoWriter object
video_writer.release()